In [1]:
from random import *
from math import *

In [125]:
class EightPuzzle():
    
    def __init__(self, solved_state, shuffles=10, initial_state=None):
        self.solved_state = solved_state
        self.state = None
        
        if initial_state:
            self.state = initial_state
        elif shuffles and not initial_state:
            self.state = self.shuffle(shuffles)
    
    @staticmethod
    def row(position):
        return floor(position/3)
    
    @staticmethod
    def col(position):
        return position % 3
    
    @staticmethod
    def index(row, col):
        return row * 3 + col
    
    def get_moves(self, state):
        space = state.index(0)
        r = self.row(space)
        c = self.col(space)
        
        moves = []
        if r == 0:
            moves.append("S")
        elif r == 1:
            moves += ["N", "S"]
        else:
            moves.append("N")
        if c == 0:
            moves.append("E")
        elif c == 1 :
            moves += ["E", "W"]
        else:
            moves.append("W")
        return moves
    
    def do_move(self, state, move, prnt = False):
        new_state = list(state)
        space = new_state.index(0)
        r = self.row(space)
        c = self.col(space)
        
        if move == "N":
            move_index = self.index(r-1,c)
        elif move == "S":
            move_index = self.index(r+1,c)
        elif move == "E":
            move_index = self.index(r, c+1)
        elif move == "W":
            move_index = self.index(r, c-1)
    
        if prnt:
            print(f'row = {r} | col = {c} | move_index = {move_index}')

        new_state[space], new_state[move_index] = state[move_index], state[space]
        return new_state
    
    def undo_move(self, state, move):
        if move == "N":
            return self.do_move(state,"S")
        elif move == "S":
            return self.do_move(state, "N")
        elif move == "E":
            return self.do_move(state, "W")
        elif move == "W":
            return self.do_move(state, "E")
        
    def energy(self):
        energy = 0
        for i, val in enumerate(self.state):
            if val != 0:
                solved_i = self.solved_state.index(val)
                distance = abs(i - solved_i)
                energy += distance
        return energy
    
    def is_solved(self):
        return self.energy() == 0
    
    def simulated_anneal(self, initial_state = None, solved_state = None, T = 100, p = 1):
        curr_energy = self.energy()
        prev_energy = self.energy()
        orig_T = T
        
        while not self.is_solved() and T > 0:
            moves = self.get_moves(self.state)
            accept = False
            
            while not accept and T > 0:
                move = choice(moves)
                self.state = self.do_move(self.state, move)
                curr_energy = self.energy()
                change_energy = curr_energy - prev_energy
                
                if change_energy <= 0:
                    accept = True
                else:
                    boltzmann = exp(-float(p*change_energy) / T)
                    n = random()
                    if n <= boltzmann:
                        accept = True
                        
                    if not accept:
                        self.undo_move(self.state, move)
                        
                prev_energy = curr_energy
                T -= 1
                
        if self.is_solved():
            print(f"Puzzle solved in {orig_T - T} moves with probability {p}")
        else:
            print(f"Puzzle Couldn't be solved in {orig_T} moves with probability {p}")
        
    def shuffle(self, shuffles = 10):
        st = list(self.solved_state)
        for i in range(shuffles):
            move = choice(self.get_moves(st))
            st = self.do_move(st, move)
        return st
        

    @staticmethod
    def print_puzzle(state, as_str=False):
        msg = f"""
        {state[0]} {state[1]} {state[2]}
        {state[3]} {state[4]} {state[5]}
        {state[6]} {state[7]} {state[8]}
        """
        if as_str:
            return msg
        print(msg)
        
    
    
    

In [133]:
solved_state = [1,2,3,4,5,6,7,8,0]
puzzle = EightPuzzle(solved_puzzle, shuffles=5)

print(f"""
Initial State
--------------
{puzzle.print_puzzle(puzzle.state, as_str=True)}

Goal State
--------------
{puzzle.print_puzzle(puzzle.solved_state, as_str=True)}
""")


puzzle.simulated_anneal(T = 100)


print(f"""
State
--------------
{puzzle.print_puzzle(puzzle.state, as_str=True)}

Goal State
--------------
{puzzle.print_puzzle(puzzle.solved_state, as_str=True)}
""")


puzzle.is_solved()


Initial State
--------------

        1 2 3
        4 5 6
        7 0 8
        

Goal State
--------------

        1 2 3
        4 5 6
        7 8 0
        

Puzzle solved in 3 moves with probability 1

State
--------------

        1 2 3
        4 5 6
        7 8 0
        

Goal State
--------------

        1 2 3
        4 5 6
        7 8 0
        



True